In [40]:
import datetime as dt
import pandas as pd
import psycopg2
import pytz
import re
import sys
import time
import warnings
from bs4 import BeautifulSoup
from io import StringIO
from psycopg2.extras import execute_values
from psycopg2.extensions import AsIs
from typing import List
from urllib.request import urlopen
import sys
sys.path.insert(0, '../..')
from db_utils import generate_unique_game_id

In [5]:
url = "https://www.sports-reference.com/cfb/boxscores/index.cgi?month=1&day=2&year=2023"
html = urlopen(url)
soup = BeautifulSoup(html, features="html.parser")

In [7]:
# # soup.find_all("table", {"class": re.compile("teams")})
# tables = soup.find_all("table", {"class": re.compile("teams")})
# html_string = "\n".join(str(table) for table in tables)
# html_io = StringIO(html_string)


    # pattern = "^[A-Za-z]{3} \d{1,2}/\d{2}$"
    # match_string = df.iloc[0][0]
    # if re.fullmatch(pattern, match_string):
    #     match_string.to_strftime()

    # home_row = df.iloc[1]
    # home, home_rank = get_rank_from_row(home_row)
    # visitor_row = df.iloc[2]
    # visitor, visitor_rank = get_rank_from_row(visitor_row)

In [6]:
def is_valid_date(date_string):
    # Regex pattern for "Month Day, Year" with an optional day of the week
    pattern = r'^(?:\w{3,9},\s)?\w{3,9}\s\d{1,2},\s\d{4}$'
    return bool(re.match(pattern, date_string))

def get_rank_from_row(row):
    pattern2 = r"^(.*?)\s\((\d{1,2})\)$"
    university = row[0]
    match = re.fullmatch(pattern2, university)
    if match:
         return match.group(1), int(match.group(2))
    else:
         return university, None

cand_divs = soup.find_all(attrs={"class": re.compile("game_summaries")})
date, div = None, None
date_format = "%A, %B %d, %Y"


for cand_div in cand_divs:
    h2_text = cand_div.find('h2').text
    if is_valid_date(h2_text):
        date = dt.datetime.strptime(h2_text, date_format)
        html_io = StringIO(str(cand_div))
        all_games_table = pd.read_html(html_io)

# have the desired table, no "other games this week"

In [10]:
# html_string = "\n".join(str(table) for table in tables)
# html_io = StringIO(html_string)

# all_games_table = pd.read_html(html_io, header=0)[0]

In [136]:
# [div for div in tables if div.find_parent('div', {"class": re.compile("game_summaries")})]

In [54]:
cand_divs = soup.find_all(attrs={"class": re.compile("game_summaries")})
date_format = "%A, %B %d, %Y"

for cand_div in cand_divs:
    h2_text = cand_div.find('h2').text
    if is_valid_date(h2_text):
        date = dt.datetime.strptime(h2_text, date_format)
        html_io = StringIO(str(cand_div))
        all_games_table = pd.read_html(html_io)
        break


for cand_div in cand_divs:
    h2_text = cand_div.find('h2').text
    if is_valid_date(h2_text):
        date = dt.datetime.strptime(h2_text, date_format)
        html_io = StringIO(str(cand_div))
        all_games_table = pd.read_html(html_io)
        
def get_rank_from_row(row):
    pattern2 = r"^(.*?)\s\((\d{1,2})\)$"
    university = row[0]
    match = re.fullmatch(pattern2, university)
    if match:
         return match.group(1), int(match.group(2))
    else:
         return university, None
    

def clean_df(df: pd.DataFrame, date: dt.datetime): 
    pattern = "^[A-Za-z]{3} \d{1,2}/\d{2}$"
    match_string = df.iloc[0][0]
    if re.fullmatch(pattern, match_string):
        match_string.to_strftime()

    home_row = df.iloc[1]
    home, home_rank = get_rank_from_row(home_row)
    home_points = home_row[1]
    visitor_row = df.iloc[2]
    visitor, visitor_rank = get_rank_from_row(visitor_row)
    visitor_points = visitor_row[1]
    
    id_row = pd.Series({
        'Date': [date],
        'Home': [home],
        'Visitor': [visitor]
    })
    
    return pd.DataFrame({
        'date': [date],
        'home': [home],
        'home_points': [home_points],
        'visitor': [visitor],
        'visitor_points': [visitor_points],
        'home_rank': [home_rank],
        'visitor_rank': [visitor_rank],
        'unique_id': [generate_unique_game_id(id_row)]
    })

In [55]:
game_list = []
for game in all_games_table:
    game_list.append(clean_df(game, date))

['Mississippi State']
['Penn State']
['LSU']
['Tulane']


In [39]:
pd.concat(game_list)

,home,home_points,visitor,visitor_points,home_rank,visitor_rank
0,Mississippi State,19,Illinois,10,24,None
0,Penn State,35,Utah,21,9,7
0,LSU,63,Purdue,7,16,None
0,Tulane,46,USC,45,14,8


In [35]:
temp = all_games_table[1]
# temp.drop_index(inplace=True)
clean_df(temp)

,home,home_points,visitor,visitor_points,home_rank,visitor_rank
0,Penn State,35,Utah,21,9,7


In [98]:
all_games_table

[          0   1      2
 0  USC (23)  27  Final
 1  LSU (13)  20    NaN,
                     0        1        2
 0             Mon 9/2  Mon 9/2  Mon 9/2
 1      Boston College       28    Final
 2  Florida State (10)       13      NaN,
            0         1         2
 0   Sat 8/31  Sat 8/31  Sat 8/31
 1  Merrimack         6     Final
 2  Air Force        21       NaN,
                   0         1         2
 0          Sat 8/31  Sat 8/31  Sat 8/31
 1  Western Kentucky         0     Final
 2              None      None      None,
    index                     0         1         2
 0      0              Sat 8/31  Sat 8/31  Sat 8/31
 1      1  East Tennessee State        10     Final
 2      2     Appalachian State        38       NaN,
               0         1         2
 0      Sat 8/31  Sat 8/31  Sat 8/31
 1    New Mexico        39     Final
 2  Arizona (21)        61       NaN,
                0         1         2
 0       Sat 8/31  Sat 8/31  Sat 8/31
 1        Wyoming         

In [87]:
def get_rank_from_row(row):
    pattern2 = r"^(.*?)\s\((\d{1,2})\)$"
    university = row[0]
    match = re.fullmatch(pattern2, university)
    if match:
         return match.group(1), int(match.group(2))
    else:
         return university, None

In [97]:
get_rank_from_row(all_games_table[5].iloc[2])

('Arizona', '21')

In [94]:
all_games_table[5]

,0,1,2
0,Sat 8/31,Sat 8/31,Sat 8/31
1,New Mexico,39,Final
2,Arizona (21),61,NaN


In [79]:
temp.iloc[0][0]

'Western Kentucky'

In [86]:
regex2 = r".*\(\b([1-9]|1[0-9]|2[0-5])\b\)"
re.fullmatch(regex2, "Western Kentucky (5)")

<re.Match object; span=(0, 20), match='Western Kentucky (5)'>

In [75]:
temp = all_games_table[3][1:]
temp.loc[:, "rank"] = None
# temp

/var/folders/z_/5kxjclhd5090czxnlnxslydr0000gn/T/ipykernel_88211/1111760236.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.loc[:, "rank"] = None


regex = "^[A-Za-z]{3} \d{1,2}/\d{2}$"
string = all_games_table[3].iloc[0][0]

s = re.fullmatch(regex, string)

In [64]:
s

<re.Match object; span=(0, 8), match='Sat 8/31'>

In [ ]:
month = month.lower()

    # URL to scrape, notice f string:
    url = (
        f"https://www.basketball-reference.com/leagues/NBA_%(year)s_games-%(month)s.html"
        % {"year": year, "month": month}
    )
    try:
        html = urlopen(url)
    except Exception as e:
        print(e)
        print(
            "Failure: %(month)s, %(year)s all_games URL not pulled"
            % {"month": month.capitalize(), "year": year}
        )
        return None

    # Kicks you out if you request over 20 times over a minute
    time.sleep(3.5)

    soup = BeautifulSoup(html, features="html.parser")

    # Convert the tables into a string and wrap it with StringIO
    tables = soup.find_all("table", {"id": re.compile("schedule")})
    html_string = "\n".join(str(table) for table in tables)
    html_io = StringIO(html_string)

    all_games_table = pd.read_html(html_io, header=0)[0]
    all_games_table = clean_all_games_table(all_games_table)
    if all_games_table is not None:
        all_games_table["Unique_ID"] = all_games_table.apply(
            generate_unique_game_id, axis=1
        )
    return all_games_table